In [ ]:
!pip install torch torchvision torchaudio

# torch 가 정상적으로 설치되었는지 확인하기

In [1]:
import torch
print(torch.__version__)  # 설치된 PyTorch 버전 출력
print(torch.cuda.is_available())  # GPU 사용 가능 여부 출력 (True면 GPU 사용 가능)

2.6.0
False


# AutoModel 클래스로 id에 맞는 모델 가져오기

model_id를 허깅페이스 모델 허브의 저장소 경로 혹은 로컬 경로 지정해서 모델 불러올 수 있다.
RoBERTa는 구글의 BERT를 개선한 모델이고, 여기서 사용하는 모델은 RoBERTa 모델을 한국어로 학습한 모델이다.

In [2]:
from transformers import AutoModel

model_id = 'klue/roberta-base'
model = AutoModel.from_pretrained(model_id)


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## AutoModel 클래스
어떻게 `klue/roberta-base` 저장소의 모델이 RoBERTa 계열의 모델인지 알 수 있을까?
- 허깅페이스 모델을 저장할 때 config.json 파일이 함께 저장되는데 해당 파일에는 model_type, 여러 설정 파라미터(num_attention_heads 등), 어휘 사전 크기(vocab_size), tokenizer_class 등이 저장됨

AutoModel과 AutoTokenizer 클래스는 config.json을 참고해 적절한 모델과 토크나이저를 불러온다.

# 텍스트 분류 헤드가 붙은 모델 불러오기

`SamLowe/roberta-base-go_emotions` 모델: 분류 헤드가 포함되어 있으며 입력 문장이 어떤 감성을 나타내는지 분류함(ex: admiration, amusement, anger etc...)
`AutoModelForSequenceClassification`: 텍스트 시퀀스 분류를 위한 헤드가 포함된 모델을 불러올 때 사용하는 클래스

In [3]:
from transformers import AutoModelForSequenceClassification

model_id = 'SamLowe/roberta-base-go_emotions'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

# 텍스트 분류를 위한 아키텍처에 모델 바디만 불러오기
AutoModelForSequenceClassification 클래스를 사용하면 분류 헤드가 붙은 모델을 불러올 수 있다.
이 클래스를 사용해 모델 바디 부분의 파라미터만 있는 `klue/roberta-base` 모델을 불러온다면, 아래와 같은 경고가 발생한다.
- ⚠️경고 내용: 모델의 바디 부분은 `klue/roberta-base`의 사전학습된 파라미터를 불러왔으나 `klue/roberta-base` 모델 허브에서는 분류 헤드에 대한 파라미터를 찾을 수 없어 랜덤!!으로 초기화 했다는 내용
- ⚠️분류 헤드가 랜덤으로 초기화되었기 때문에 그대로 사용하면 안되고 추가 학습 이후에 사용하라고 안내됨

분류 헤드가 학습되지 않았기 때문에 의미 있는 분류를 할 수 없음, 그렇다면 어떻게 분류 헤드를 학습해 분류 모델을 만들 수 있을까?

In [4]:
from transformers import AutoModelForSequenceClassification

model_id = 'klue/roberta-base'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 토크나이저 활용

⭐️토크나이저: 텍스트를 토큰 단위로 나누고 각 토큰을 대응하는 토큰 아이디로 변환, 필요한경우 특수 토큰을 추가하는 역할
토크나이저도 학습 데이터를 통해 어휘 사전을 구축하므로 일반적으로 모델과 함께 저장됨

허깅페이스 허브에서 모델과 토크나이저를 불러올때 동일한 모델 아이디로 맞춰야함!!

- tokenizer_config.json: 토크나이저의 종류나 설정에 대한 정보를 갖고 있음
- tokenizer.json: 실제 어휘 사전 정보를 갖고 있음

In [5]:
from transformers import AutoTokenizer

model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

## 토크나이저 예제

tokenizer("문장")
- input_ids: 토큰 아이디의 리스트
- attention_mask: 토큰이 실제 텍스트인지 아니면 길이를 맞추기 위해 추가한 padding인지 알려줌
- token_type_ids: 토큰이 속한 문장의 아이디를 알려줌

In [10]:
tokenized = tokenizer("토크나이저는 텍스트를 토큰 단위로 나눈다")

for k, v in tokenized.items():
    print(k, v)

input_ids [0, 9157, 7461, 2190, 2259, 8509, 2138, 1793, 2855, 5385, 2200, 20950, 2]
token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


- input_ids: 토큰화했을 때 각 토큰이 토크나이저 사전의 몇번째 항목인지 나타냄
input_ids에서 첫번째 항목은 0이고 두번째 항목은 9157인데, 각각 [CLS]와 '토크'에 대응되는 것을 확인할 수 있음
- attention_mask[idx] == 1: 실제 토큰 else 패딩

token_type_ids[idx] == 0: 일반적으로 첫번째 문장임

In [9]:
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))

['[CLS]', '토크', '##나이', '##저', '##는', '텍스트', '##를', '토', '##큰', '단위', '##로', '나눈다', '[SEP]']


토큰 아이디를 다시 텍스트로 돌리고 싶다면 토크나이저의 `decode` 메서드 사용
- 특수 토큰이 추가됨을 확인할 수 있음
    - [CLS]
    - [SEP]
- skip_special_tokens=True 시 특수 토큰 스킵

In [11]:
print(tokenizer.decode(tokenized['input_ids']))
print(tokenizer.decode(tokenized['input_ids'], skip_special_tokens=True))

[CLS] 토크나이저는 텍스트를 토큰 단위로 나눈다 [SEP]
토크나이저는 텍스트를 토큰 단위로 나눈다


In [12]:
# 토크나이저에 여러 문장을 넣을수도 있다.
tokenizer(['안녕하세요?', 'Hello, world'])

{'input_ids': [[0, 5891, 2205, 5971, 35, 2], [0, 13472, 10211, 2036, 16, 30520, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

한 번에 여러 문장을 모델에 넣어야 하는 경우가 있음

예를 들어, 2개의 문장이 서로 원인/결과 관계인지 학습시키고 싶을 때
- 이 경우, 2개의 문장이 하나의 데이터라는 것을 표시하기 위해 아래처럼 리스트로 한번 더 감싼다

In [13]:
tokenizer([['안녕하세요?', 'Hello, world']])

{'input_ids': [[0, 5891, 2205, 5971, 35, 2, 13472, 10211, 2036, 16, 30520, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

tokenizer의 batch_decode() 메서드
- input_ids 부분의 토큰 아이디를 문자열로 복원
- 2개의 문장을 한번에 토큰화할 경우, [SEP]으로 두 문장을 구분함
- 특수 토큰은 모델의 아키텍처에 따라 달라질 수 있음

In [14]:
first_tokenized_result = tokenizer(['첫번째 문장', '두번째 문장'])['input_ids']
tokenizer.batch_decode(first_tokenized_result)

['[CLS] 첫번째 문장 [SEP]', '[CLS] 두번째 문장 [SEP]']

In [15]:
second_tokenized_result = tokenizer([['첫번째 문장', '두번째 문장']])['input_ids']
tokenizer.batch_decode(second_tokenized_result)

['[CLS] 첫번째 문장 [SEP] 두번째 문장 [SEP]']

### token_type_ids: 문장을 구분하는 역할

BERT는 학습할때 2개의 문장이 서로 이어지는지 맞추는 NST(Next Sentence prediction) 작업을 활용, 이를 위해 문장을 구분하는 토큰 타입 아이디를 만듦
그래서 BERT의 토크나이저를 불러오면, 문장에 따라 토큰 타입 아이디를 구분함

klue/bert-base 토크나이저를 사용하면 첫번째 문장의 토큰 타입 아이디는 0, 두번째 문장의 토큰 타입 아이디는 1
klue/roberta-base의 경우, 모두 0임 -> RoBERTa 계열 모델은 NSP 작업을 학습 과정에서 제거해서 문장 토큰 구분이 필요없다.

In [16]:
bert_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
bert_tokenizer([['첫번째 문장', '두번째 문장']])

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

{'input_ids': [[2, 1656, 2517, 3135, 6265, 3, 864, 2517, 3135, 6265, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [18]:
roberta_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
roberta_tokenizer([['첫번째 문장', '두번째 문장']]) # token_type_ids 가 모두 0 이다.

{'input_ids': [[0, 1656, 2517, 3135, 6265, 2, 864, 2517, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [19]:
# roberta-base 토크나이저로 영어 문장을 토큰화하면 결과에 token_type_ids 항목이 없다.
en_roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
en_roberta_tokenizer([['first sentence', 'second sentence']])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'input_ids': [[0, 9502, 3645, 2, 2, 10815, 3645, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}

### attention_mask
> 해당 토큰이 패딩 토큰인지 실제 데이터인지에 대한 정보를 담음

패딩: 모델에 입력하는 토큰 아이디의 길이를 맞추기 위해 추가하는 특수 토큰
- tokenizer의 padding='longest' 를 입력하면 입력한 문장 중 가장 긴 문장에 맞춰 패딩 토큰 추가

In [24]:
"""
두 번째 문장이 더 길기 때문에 더 긴 문장에 맞춰 패딩 토큰을 추가한다면 첫 번째 문장에 패딩이 추가됨
input_ids에서 첫 번째 문장에 패딩 토큰(토큰 아이디 = 1)이 6개 추가됨
attention_mask에는 패딩 토큰을 나타내는 숫자 0이 6개 붙음
"""
result = tokenizer(['첫 번째 문장은 짧다.', '두 번째 문장은 첫 번째 문장보다 더 길다.'], padding='longest')
for k, v in result.items():
    print(f'{k}: {v}')

input_ids: [[0, 1656, 1141, 3135, 6265, 2073, 1599, 2062, 18, 2, 1, 1, 1, 1, 1, 1, 1], [0, 864, 1141, 3135, 6265, 2073, 1656, 1141, 3135, 6265, 2178, 2062, 831, 647, 2062, 18, 2]]
token_type_ids: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


# datasets 라이브러리 활용하기

In [25]:
from datasets import load_dataset

# load_dataset(): 데이터셋을 불러오는 함수(데이터셋 이름, 서브셋 이름)
klue_mrc_dataset = load_dataset('klue', 'mrc')

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [26]:
klue_mrc_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})

In [27]:
# 유형이 train인 데이터만 보고 싶은 경우
print(load_dataset('klue', 'mrc', split='train'))

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})


#### 로컬의 데이터를 활용하는 법

In [29]:
from datasets import load_dataset, Dataset

# local의 csv - 로컬에 해당 데이터가 없어서 오류남 주석처리
# dataset = load_dataset('csv', data_files="file.csv")

# 파이썬 딕셔너리 
my_dict = {"a": [1, 2, 3]}
dataset2 = Dataset.from_dict(my_dict)

# 판다스 데이터프레임
import  pandas as pd
df = pd.DataFrame({"a": [1, 2, 3]})
dataset3 = Dataset.from_pandas(df)

In [30]:
dataset2

Dataset({
    features: ['a'],
    num_rows: 3
})

In [31]:
dataset3

Dataset({
    features: ['a'],
    num_rows: 3
})

# 모델 학습시키기
한국어 기사 제목을 바탕으로 기사의 카테고리를 분류하는 텍스트 분류 모델 실습

✨과정
1. 데이터셋 준비
2. 모델과 토크나이저를 불러와 모델 학습 - 허깅페이스 트랜스포머에서는 학습 과정을 추상화한 Trainer API를 제공(but, 내부 과정을 알 수 없음)
3. 학습을 마친 모델을 저장하거나 공유할 수 있도록 허깅페이스 허브에 업로드

In [32]:
"klue 데이터셋의 YNAT(연합 뉴스 기사 제목과 기사가 속한 카테고리 정보 포함) 서브셋 활용"
KLUE, MNC = 'klue', 'ynat'

klue_tc_train = load_dataset(KLUE, MNC, split='train')
klue_tc_eval = load_dataset(KLUE, MNC, split='validation')
klue_tc_train

train-00000-of-00001.parquet:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 45678
})

In [38]:
"""
guid: 데이터의 고유 ID - 불필요
title
label: 속한 카테고리 ID
url - 불필요
date - 불필요
"""
klue_tc_eval

Dataset({
    features: ['guid', 'title', 'label', 'url', 'date'],
    num_rows: 9107
})

In [34]:
klue_tc_train[0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [37]:
"데이터셋의 정보를 저장하고 있는 features 속성에서 label 컬럼의 항목별 이름 확인"
klue_tc_train.features['label'].names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [39]:
"실습에 사용되지 않는 불필요한 컬럼 제거"
removed_columns = ['guid', 'url', 'date']
klue_tc_train = klue_tc_train.remove_columns(removed_columns)
klue_tc_eval = klue_tc_eval.remove_columns(removed_columns)
klue_tc_train

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})

카테고리를 확인하기 쉽게 label_str 컬럼 추가
features 속성에서 label 컬럼을 확인하면 레이블id 와 카테고리를 연결할 수 있는 ClassLabel 객체가 있음
해당 객체에는 Id를 카테고리로 변환하는 int2str 메서드가 있음
- int2str(1): '경제' 카테고리 반환


In [43]:
klue_tc_label = klue_tc_train.features['label']
klue_tc_label

ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

In [42]:
klue_tc_train.features['label'].int2str(1)

'경제'

In [44]:
def make_str_label(batch):
    batch['label_str'] = klue_tc_label.int2str(batch['label'])
    return batch

In [45]:
klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

In [51]:
klue_tc_train[:5]

{'title': ['유튜브 내달 2일까지 크리에이터 지원 공간 운영',
  '어버이날 맑다가 흐려져…남부지방 옅은 황사',
  '내년부터 국가RD 평가 때 논문건수는 반영 않는다',
  '김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것',
  '회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간'],
 'label': [3, 3, 2, 2, 3],
 'label_str': ['생활문화', '생활문화', '사회', '사회', '생활문화']}

In [52]:
len(klue_tc_train)

45678

### 학습/검증/테스트 데이터셋 분할
학습 데이터 중 10_000개만 추출해 사용, train_test_split() 메서드에 test_size를 입력하여 학습 데이터셋과 테스트 데이터셋으로 분리
학습이 잘되고 있는지 확인할 검증 데이터와 성능 확엔에 사용할 테스트 데이터는 검증 데이터셋(klue_tc_eval)에서 각각 1000개씩 뽑아 사용

In [54]:
train_dataset = klue_tc_train.train_test_split(test_size=10_000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)

test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test']

### Trainer API

허깅페이스는 학습에 필요한 다양한 기능(데이터로더 준비, 로깅, 평가, 저장 등)을 `TrainingArguments` 만으로 쉽게 활용할 수 있는 트레이너 API 제공